나만의 모델로 구성해보자

In [0]:
import torch
import torchvision 

In [0]:
trans = torchvision.transforms.Compose([
                                              torchvision.transforms.ToTensor()
])

#data 가져오기
train_dataset = torchvision.datasets.FashionMNIST('~/.pytorch/F_MNIST_data/', download=True, transform=trans)
val_dataset = torchvision.datasets.FashionMNIST('~/.pytorch/F_MNIST_data/', download=True, transform=trans)
test_dataset = torchvision.datasets.FashionMNIST('~/.pytorch/F_MNIST_data/', download=False, transform=trans)

indices = list(range(len(train_dataset.train_data)))
split = int(len(indices)*0.2)
train_idx, val_idx = indices[:split], indices[split:]

train_sampler = torch.utils.data.sampler.SubsetRandomSampler(train_idx)
train_loader = torch.utils.data.DataLoader(train_dataset,batch_size=32, sampler=train_sampler)
val_sampler = torch.utils.data.sampler.SubsetRandomSampler(val_idx)
val_loader = torch.utils.data.DataLoader(val_dataset,batch_size=32, sampler=val_sampler)
test_loader = torch.utils.data.DataLoader(test_dataset,batch_size=32)


  0%|          | 16384/26421880 [00:00<03:05, 142095.13it/s]

26427392it [00:00, 70037422.10it/s]                            


Extracting /root/.pytorch/F_MNIST_data/FashionMNIST/raw/train-images-idx3-ubyte.gz to /root/.pytorch/F_MNIST_data/FashionMNIST/raw


32768it [00:00, 462955.38it/s]
  2%|▏         | 98304/4422102 [00:00<00:04, 890108.31it/s]

Extracting /root/.pytorch/F_MNIST_data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to /root/.pytorch/F_MNIST_data/FashionMNIST/raw


4423680it [00:00, 22777028.74it/s]                         
8192it [00:00, 157901.76it/s]


Extracting /root/.pytorch/F_MNIST_data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to /root/.pytorch/F_MNIST_data/FashionMNIST/raw
Extracting /root/.pytorch/F_MNIST_data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to /root/.pytorch/F_MNIST_data/FashionMNIST/raw
Processing...
Done!


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:53: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")


In [0]:
from torch import nn

Fashion data는 28*28 grayscale 이미지를 사용하기 때문에
Convolutional layer를 사용하는 것이 더 좋은 성능을 낼 것이라고 생각함.

모델 구성

1. conv
2. maxpool
3. conv
4. maxpool
5. conv
6. fc
7. fc


In [0]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 128, 3)
        self.conv2 = nn.Conv2d(128, 256, 3, stride=2)
        self.conv3 = nn.Conv2d(256, 64, 3)
        self.maxpool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64,128)
        self.fc2 = nn.Linear(128,10)

    def forward(self, x):
        x = self.maxpool(nn.functional.relu(self.conv1(x)))
        x = self.maxpool(nn.functional.relu(self.conv2(x)))
        x = nn.functional.relu(self.conv3(x))
        x = x.view(x.size()[0],-1)
        x = nn.functional.relu(self.fc1(x))
        x = nn.functional.softmax(self.fc2(x))

        return x

Cost 함수 정의

분류문제에서는 Cross entropy가 주로 loss로 사용된다.

In [0]:
model = Net()
criterion = nn.CrossEntropyLoss()

Optimizer 정의

Optimizer로 Adam 정의해보자. learning Rate는 0.03으로 했을 때 잘 안됐음.
0.001로 하니까 잘 줄어든다.

In [0]:
optimizer = torch.optim.Adam(model.parameters())

In [0]:
MAX_EPOCH = 2
for epoch in range(MAX_EPOCH):
    total_loss = 0
    step = 0
    for X, y in train_loader:
        step+=1
        optimizer.zero_grad()   #gradient 초기화
        output = model(X)
        loss = criterion(output,y)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        if step%20==0:
            print('{}, {} : {}'.format(epoch,step, total_loss/20))
            total_loss = 0

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


0, 20 : 2.267293119430542
0, 40 : 2.1481494069099427
0, 60 : 1.9837896466255187
0, 80 : 1.9087730765342712
0, 100 : 1.8562082648277283
0, 120 : 1.8177162647247314
0, 140 : 1.8919919431209564
0, 160 : 1.8502901554107667
0, 180 : 1.8724698960781097
0, 200 : 1.8553234100341798
0, 220 : 1.855963295698166
0, 240 : 1.8490013420581817
0, 260 : 1.8560051023960114
0, 280 : 1.823264718055725
0, 300 : 1.8354657113552093
0, 320 : 1.841720986366272
0, 340 : 1.8405284702777862
0, 360 : 1.85869500041008
1, 20 : 1.826810747385025
1, 40 : 1.8035564661026
1, 60 : 1.8116839468479156
1, 80 : 1.8422003865242005
1, 100 : 1.870272022485733
1, 120 : 1.8253928959369659
1, 140 : 1.8097397267818451
1, 160 : 1.7999345183372497
1, 180 : 1.807234799861908
1, 200 : 1.7911263227462768
1, 220 : 1.8267874479293824
1, 240 : 1.8109180629253387
1, 260 : 1.8226750314235687
1, 280 : 1.8089794039726257
1, 300 : 1.802241039276123
1, 320 : 1.7948085904121398
1, 340 : 1.7379481136798858
1, 360 : 1.7376123726367951


In [0]:
total_loss = 0
step = 0
total = 0
correct = 0
with torch.no_grad():
    for val_x, val_y in val_loader:
        step+=1
        val_output = model(val_x)
        v_loss = criterion(val_output, val_y)
        total_loss += v_loss.item()
        _, predicted = torch.max(val_output.data, 1)
        total += val_y.size(0)
        correct += (predicted == val_y).sum().item()
        if step%100 == 0:
            print('{} : {}'.format(step, correct/total))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


100 : 0.7171875
200 : 0.7153125
300 : 0.719375
400 : 0.721640625
500 : 0.7196875
600 : 0.72140625
700 : 0.7208928571428571
800 : 0.7209765625
900 : 0.7204513888888889
1000 : 0.72053125
1100 : 0.7213636363636363
1200 : 0.7219791666666666
1300 : 0.7218990384615385
1400 : 0.7222544642857143
1500 : 0.722125


In [0]:
step = 0
total= 0
correct = 0
total_loss = 0
with torch.no_grad():
    for test_x, test_y in test_loader:
        step += 1
        test_output = model(test_x)
        test_loss = criterion(test_output, test_y)
        total_loss += test_loss.item()
        _, predicted = torch.max(test_output.data, 1)
        total += test_y.size(0)
        correct += (predicted == test_y).sum().item()
        if step%100 == 0:
            print('test data loss : {}, correct : {}'.format(total_loss/100, correct/total))
            total_loss = 0

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


test data loss : 1.7269757461547852, correct : 0.7353125
test data loss : 1.723120037317276, correct : 0.7375
test data loss : 1.7412415611743928, correct : 0.7317708333333334
test data loss : 1.7469896376132965, correct : 0.727890625
test data loss : 1.7392031407356263, correct : 0.7271875
test data loss : 1.7460145938396454, correct : 0.7252604166666666
test data loss : 1.7455536365509032, correct : 0.7240625
test data loss : 1.739712927341461, correct : 0.723515625
test data loss : 1.7488032591342926, correct : 0.7222916666666667
test data loss : 1.7372376036643982, correct : 0.722625
test data loss : 1.735630648136139, correct : 0.7229261363636363
test data loss : 1.7391197896003723, correct : 0.7227864583333333
test data loss : 1.7384938085079193, correct : 0.7229567307692307
test data loss : 1.745896555185318, correct : 0.7224553571428571
test data loss : 1.7324040949344635, correct : 0.7229166666666667
test data loss : 1.7198126006126404, correct : 0.7241015625
test data loss : 

정확도 72% 정도